In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Add project 1 to sys path 
import sys
sys.path.append('../project1/')
import tools as proj1_tools
import project2_tools as proj2_tools

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
L = 40
temps = proj2_tools.get_available_t()
states = proj2_tools.read_t(temps[0])
#states.shape

# Two dimensional Ising Model 

Make training data

In [ ]:
def get_train_data(n, L,t_crit):
    """Pick some random training data from whole data set"""
    Nt = temps.size
    Nl = L * L
    data = np.zeros((Nt, n, Nl),dtype=np.int8)
    for i,t in enumerate(temps):
        d = proj2_tools.read_t(t)
        d[np.where(d == 0)] = -1
        data[i] = d
        
    n_ordered = np.sum(temps > t_crit)
    classes = np.zeros((Nt,n),dtype = np.int8)
    classes[:-n_ordered] = 1
    
    data = data.reshape(-1, Nl)
    
    classes = classes.ravel()
    return data, classes

    
n = 10000
L = 40
data,classes = get_train_data(n, L, 2.3)

In [ ]:
N_train = 1000

np.random.seed(0)
indx = np.arange(data.shape[0])
np.random.shuffle(indx)

X_train = data[indx][:N_train]
X_train = np.c_[np.ones_like(X_train[:,0]), X_train]
y_train = classes[indx][:N_train]

X_test = data[indx][N_train:2*N_train]
X_test = np.c_[np.ones_like(X_test[:,0]), X_test]
y_test = classes[indx][N_train:2*N_train]

X_test2 = data[indx][2*N_train:3*N_train]
X_test2 = np.c_[np.ones_like(X_test2[:,0]), X_test2]
y_test2 = classes[indx][2*N_train:3*N_train]

X_test3 = data[indx][3*N_train:4*N_train]
X_test3 = np.c_[np.ones_like(X_test3[:,0]), X_test3]
y_test3 = classes[indx][3*N_train:4*N_train]

In [ ]:
temps

# Logistic regression
##### Compared with $sklearn.linear\_model.LogisticRegression$

In [ ]:
import logreg
from sklearn.linear_model import LogisticRegression

#### Prepare test sets and variables:

In [ ]:
Xtestset = [X_test,X_test2,X_test3]
ytestset = [y_test,y_test2,y_test3]

lmbd=1
iterations = 1000

#### Train/Test of our model

In [ ]:
regr = logreg.Logistic_Regression(learn_rate=0.1, iterations=iterations, normalize=False, lmbd=lmbd)
regr.fit(X_train,y_train)

Statistics:

In [ ]:
# Train
z_train = regr.predict(X_train,binary=False)
acc_train1 = regr.accuracy(y_train,z_train)
mse_train1 = np.sum(np.abs(z_train-y_train)**2)/y_train.shape[0]
# Test
acc_test1 = []
mse_test1 = []
for i,test in enumerate(Xtestset):
    z_test = regr.predict(test,binary=False)
    acc_test1.append(regr.accuracy(ytestset[i],z_test))
    mse_test1.append(np.sum(np.abs(ytestset[i]-z_test)**2)/ytestset[i].shape[0])

#### Train/Test of sklearn model:

In [ ]:
Lregr = LogisticRegression(random_state=1,verbose=0,max_iter=iterations,tol=1E-5, solver='lbfgs')
Lregr.fit(X_train,y_train)

Statistics:

In [ ]:
# Train
L_z_train = Lregr.predict(X_train)
acc_train2 = regr.accuracy(y_train,L_z_train)
mse_train2 = np.sum(np.abs(L_z_train-y_train)**2)/y_train.shape[0]

# Test
acc_test2 = []
mse_test2 = []
for i,test in enumerate(Xtestset):
    L_z_test = Lregr.predict(test)
    acc_test2.append(Lregr.score(test,ytestset[i]))
    mse_test2.append(np.sum(np.abs(ytestset[i]-L_z_test)**2)/ytestset[i].shape[0])

#### Compare

In [ ]:
print('   {:<10}{:<15}{:<15}'.format('','Our model', 'sklearn model'))
print('\nTrain set :\n')
print('   {:<10}{:<15}{:<15}'.format('Accuracy',acc_train1,acc_train2))
print('   {:>8}  {:<15}{:<15}'.format('MSE',mse_train1,mse_train2))
for i in range(len(acc_test1)):
    print('\nTest set {}:\n'.format(i+1))
    #print('{:<10}{:<15}{:<15}'.format('','Our model', 'sklearn model'))
    print('   {:<10}{:<15}{:<15}'.format('Accuracy',acc_test1[i],acc_test2[i]))
    print('   {:>8}  {:<15}{:<15}'.format('MSE',mse_test1[i],mse_test2[i]))

In [ ]:
states.shape

In [ ]:
temps.shape

In [ ]:
data.shape

In [ ]:
classes.shape

In [ ]:
state = 150000
data[state][:].shape, classes[state]

# Predict phase

In [ ]:
num = 10000
rand = np.random.randint(0,classes.shape[0]-1,num)
pred_data = np.zeros((num,data[0][:].shape[0]+1))
state = np.zeros(num)
for i,n in enumerate(rand):
    pred_data[i][0] = 1
    pred_data[i][1:] = data[n][:]
    state[i] = classes[n]

In [ ]:
pred = regr.predict(pred_data,binary=False)
pred2 = Lregr.predict(pred_data)
print('Our average correct:     ',np.sum(np.isclose(pred,state))/num)
print('sklearn average correct: ',np.sum(np.isclose(pred2,state))/num)

In [ ]:
print(pred.astype(int))
print(pred2)
print(state.astype(int))

In [ ]:
pred_data[0].shape